### Extração de Eventos e Datas (a partir de um texto)

In [1]:
texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, 
no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" 
e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o 
computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente. 
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS 
(Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema 
operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um 
ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de 
sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, 
em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base 
de sua supremacia no mercado de sistemas operacionais para desktop.
"""

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


##### usando LangChain, conseguimos construir "Function Callings" muito mais rápido

In [3]:
class Event(BaseModel):
    """Informações sobre um evento ocorrido"""
    date: str = Field(description="Data do evento no formato YYYY-MM-DD")
    event: str = Field(description="Descrição do evento extraído do texto")
    
class EventsList(BaseModel):
    """Lista de Eventos para Extração"""
    events: List[Event] = Field(description="Conjunto de eventos encontrados no texto fornecido")
    
tool_events = convert_to_openai_function(EventsList)
tool_events

{'name': 'EventsList',
 'description': 'Lista de Eventos para Extração',
 'parameters': {'type': 'object',
  'properties': {'events': {'description': 'Conjunto de eventos encontrados no texto fornecido',
    'type': 'array',
    'items': {'description': 'Informações sobre um evento ocorrido',
     'type': 'object',
     'properties': {'date': {'description': 'Data do evento no formato YYYY-MM-DD',
       'type': 'string'},
      'event': {'description': 'Descrição do evento extraído do texto',
       'type': 'string'}},
     'required': ['date', 'event']}}},
  'required': ['events']}}

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia as frases de acontecimentos e as extraia integralmente"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_events],
                            function_call={"name":"EventsList"}))

In [5]:
chain.invoke({"input": texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"events":[{"date":"1975-04-04","event":"A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, no estado do Novo México."},{"date":"1980","event":"A Microsoft firmou um contrato significativo com a IBM para fornecer o sistema operacional para o novo PC da IBM, levando à criação do MS-DOS."},{"date":"1985","event":"A Microsoft lançou o Windows, seu sistema operacional gráfico que se tornaria a base de sua supremacia no mercado de sistemas operacionais para desktop."}]}', 'name': 'EventsList'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 426, 'total_tokens': 547, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CCn5Fe9

##### Fazendo o "Output Parser" usando JSON 

In [6]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_events], function_call={'name': 'EventsList'})
         | JsonOutputFunctionsParser())

chain.invoke({'input': texto})

{'events': [{'date': '1975-04-04',
   'event': 'A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, Novo México.'},
  {'date': '1980',
   'event': 'A Microsoft firmou um contrato significativo com a IBM para fornecer o sistema operacional para o novo PC, resultando no desenvolvimento do MS-DOS.'},
  {'date': '1985',
   'event': 'A Microsoft lançou o Windows, um sistema operacional gráfico que se tornou a base de sua supremacia no mercado de sistemas operacionais para desktop.'}]}

### Extraindo Dados da Web

In [15]:
from langchain_community.document_loaders.web_base import WebBaseLoader

#loader = WebBaseLoader("https://www.techtudo.com.br")
#loader = WebBaseLoader("https://noticias.uol.com.br")
loader = WebBaseLoader("https://economia.uol.com.br/colunas/felipe-salto")
page = loader.load()
page

[Document(metadata={'source': 'https://economia.uol.com.br/colunas/felipe-salto', 'title': 'Access Denied', 'language': 'No language found.'}, page_content='\nAccess Denied\n\nAccess Denied\n \nYou don\'t have permission to access "http://economia.uol.com.br/colunas/felipe-salto" on this server.\nReference #18.b5781002.1757165930.fae7bae\nhttps://errors.edgesuite.net/18.b5781002.1757165930.fae7bae\n\n\n')]

In [16]:
class BlogPost(BaseModel):
    """Detalhes sobre uma postagem de blog"""
    title: str = Field(description="Título da postagem no blog")
    author: str = Field(description="Nome do autor da postagem no blog")
    
class BlogSite(BaseModel):
    """Conjunto de postagens de blog de um site específico"""
    posts: List[BlogPost] = Field(description="Coleção de postagens de blog do site")
    
tool_blog = convert_to_openai_function(BlogSite)
tool_blog

{'name': 'BlogSite',
 'description': 'Conjunto de postagens de blog de um site específico',
 'parameters': {'type': 'object',
  'properties': {'posts': {'description': 'Coleção de postagens de blog do site',
    'type': 'array',
    'items': {'description': 'Detalhes sobre uma postagem de blog',
     'type': 'object',
     'properties': {'title': {'description': 'Título da postagem no blog',
       'type': 'string'},
      'author': {'description': 'Nome do autor da postagem no blog',
       'type': 'string'}},
     'required': ['title', 'author']}}},
  'required': ['posts']}}

In [18]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página os posts do blog com as informações especificadas'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))

In [19]:
chain.invoke({"input": page})

[{'title': 'Title of Blog Post 1', 'author': 'Author 1'},
 {'title': 'Title of Blog Post 2', 'author': 'Author 2'}]